In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 데이터 경로 설정
data_path = "./dataset/train/digital_img"  # 여기에는 데이터셋의 경로를 입력하세요.

# 이미지 데이터 증강을 위한 ImageDataGenerator 생성
train_datagen = ImageDataGenerator(
    rescale=1./255,  # 이미지 값을 0과 1 사이로 조정
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# 데이터셋 로딩 및 전처리
train_generator = train_datagen.flow_from_directory(
    data_path,
    target_size=(224, 224),  # ResNet50의 기본 입력 크기에 맞춤
    batch_size=32,
    class_mode='categorical',  # 다중 클래스 분류
    shuffle=True
)

# ResNet50 모델 불러오기 (include_top=False로 변경)
base_model = ResNet50(weights='imagenet', include_top=False)

# 새로운 분류 층 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(7, activation='softmax')(x)  # 클래스 수에 맞게 조정

# 새로운 모델 구성
model = Model(inputs=base_model.input, outputs=predictions)

# 미리 학습된 층은 동결 (Fine-tuning을 위한 기본 설정)
for layer in base_model.layers:
    layer.trainable = False

# 모델 컴파일
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tuning 진행
history = model.fit(train_generator, epochs=10, steps_per_epoch=len(train_generator))

Found 526 images belonging to 7 classes.
94765736/94765736 [==============================] - 9s 0us/step
Epoch 1/10
17/17 [==============================] - 30s 2s/step - loss: 2.0920 - accuracy: 0.2452
Epoch 2/10
17/17 [==============================] - 29s 2s/step - loss: 1.7450 - accuracy: 0.3555
Epoch 3/10
17/17 [==============================] - 28s 2s/step - loss: 1.5349 - accuracy: 0.4848
Epoch 4/10
17/17 [==============================] - 27s 2s/step - loss: 1.3459 - accuracy: 0.5722
Epoch 5/10
17/17 [==============================] - 28s 2s/step - loss: 1.2482 - accuracy: 0.5932
Epoch 6/10
17/17 [==============================] - 28s 2s/step - loss: 1.1568 - accuracy: 0.5989
Epoch 7/10
17/17 [==============================] - 27s 2s/step - loss: 1.0053 - accuracy: 0.7319
Epoch 8/10
17/17 [==============================] - 27s 2s/step - loss: 0.8907 - accuracy: 0.7605
Epoch 9/10
17/17 [==============================] - 28s 2s/step - loss: 0.8309 - accuracy: 0.7452
Epoch 10/10


In [28]:
import numpy as np
from tensorflow.keras.preprocessing import image

# 예측할 이미지 로드 및 전처리
image_path = "pred/pre3.jpg"  # 예측하고자 하는 이미지의 경로를 입력하세요.
img = image.load_img(image_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # 이미지를 [0, 1] 범위로 정규화

# 모델에 입력 이미지 전달하여 예측 수행
predictions = model.predict(img_array)

# 예측 결과 출력
predicted_class = np.argmax(predictions)

class_labels = list(train_generator.class_indices.keys())
predicted_label = class_labels[predicted_class]

print("Predicted label:", predicted_label)

1/1 [==============================] - 0s 76ms/step
Predicted label: foun
